In [1]:
import logging
logging.basicConfig(
#         filename='file1.log',
        level=logging.WARNING,
        format='%(asctime)s %(levelname)s:\t%(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
import re
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm_notebook
import sys
from datetime import datetime
from lxml import etree, html as lhtml
import pandas as pd
# from multiprocessing import Pool
# from multiprocessing.dummy import Pool as ThreadPool

In [5]:
def process_one_author_books(popular_main_page, site_name, f_write):
    def find_the_number_of_iterations(author_soup):
        total_pages = 50
        total_actors = 10000
        return total_pages, total_actors
    def process_main_page(author_soup, curr_address):        
        r_get = requests.get(curr_address)
        tree = lhtml.fromstring(r_get.text)
        table_el = tree.xpath('//div[@class="stat js-rum-hero"]')
        lst_el = table_el[0].xpath('//a[@style="font:100 12px arial,sans-serif"]/attribute::href')
        if books_descs is None:
            logging.error('no books found:\t{}'.format(curr_address))
            return
        for it in lst_el:
            print(site_name + it, file=f_write)
        logging.info('success:\t\t{}'.format(curr_address))  
    def process_fork_page(curr_address):
        for i in range(1, attempts_number+1):
            r_get = requests.get(curr_address)
            if (r_get.ok):
                break
            logging.error('bad get request ({}/{}):\t{}'.format(i, attempts_number, curr_address))
            time.sleep(1)
        if not (r_get.ok):
            return
        author_soup = BeautifulSoup(r_get.text, 'lxml')
        process_main_page(author_soup, curr_address)

    for i in range(1, attempts_number+1):
        r_get = requests.get(popular_main_page)
        if (r_get.ok):
            break
        logging.error('bad get request ({}/{}):\t{}'.format(i, attempts_number, popular_main_page))
        time.sleep(1)
    if not (r_get.ok):
        return
    
    author_soup = BeautifulSoup(r_get.text, 'lxml')   
    total_pages, _ = find_the_number_of_iterations(author_soup)
    pages = []
    for curr_page in range(2, total_pages+1):
        pages.append(popular_main_page + 'page/{}/'.format(str(curr_page)))    
#     with ThreadPool(processes=total_pages) as pool:
#         pool.map(process_fork_page, pages)
    process_main_page(author_soup, popular_main_page)
    for page in pages:
        process_fork_page(page)

# 1 этап. Нахождение ссылок на людей.

In [3]:
actors_links_txt = 'actors_links.txt'
popular_main_page = 'https://www.kinopoisk.ru/popular/names/day/2020-06-12/'
site_name = 'https://www.kinopoisk.ru'

attempts_number = 2

In [133]:
with open(actors_links_txt, 'w') as f_write:
    process_one_author_books(popular_main_page, site_name, f_write)
logging.info('completed')        